In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install seaborn
import seaborn as sns
cmap = sns.diverging_palette(220, 10, as_cmap=True)

In [ ]:

data = pd.read_csv('/kaggle/input/mental-health-in-tech-2016/mental-heath-in-tech-2016_20161114.csv')
data

In [ ]:
data = data.rename(columns={'What is your gender?': 'Gender' ,'What is your age?':'Age',"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?":"mental_Health"})
data = data.dropna(subset=['mental_Health','Age','Gender'])

In [ ]:
gender_values = data.Gender.value_counts().sort_values(ascending=False).to_frame()
gender_values = gender_values.rename(columns={'Gender': 'count'})
table_gender = gender_values.style.background_gradient(cmap=cmap)
table_gender

In [ ]:
# clean "Gender" column
data.Gender = data.Gender.str.lower()
male = ["nb masculine","male/genderqueer","male (trans, ftm)","male.","male 9:1 female, roughly","male", "m|","m", "male-ish", "maile","sex is male" ,"mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "cis male","i'm a man why didn't you make this a drop down question. you should of asked sex? and i would of answered yes please. seriously how much text can this take?"]
female = ["transitioned, m2f","female or multi-gender femme","female (props for making this a freeform field, though)","cis-woman","genderflux demi-girl","genderfluid (born female)","female/woman", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail","i identify as female.","female assigned at birth ","fm","genderqueer woman","female-bodied; no feelings about gender","cis female","   cis female","fem","cisgender female"]
other = ["dude","other/transfeminine", "unicorn","afab","cisdude","androgynous","mtf","bigender","genderfluid","none of your business","something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", 
         "genderqueer", "androgyne","transgender woman", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", 
         "female (trans)", "queer", "ostensibly male, unsure what that really means", "p", "a little about you","nonbinary","other","human"]
data.Gender.loc[data.Gender.isin(male)] = 'male'
data.Gender.loc[data.Gender.isin(female)] = 'female'
data.Gender.loc[data.Gender.isin(other)] = 'others'

gender_values = data.Gender.value_counts().sort_values(ascending=False).to_frame()
gender_values = gender_values.rename(columns={'Gender': 'count'})
table_gender = gender_values.style.background_gradient(cmap=cmap)
table_gender

In [ ]:
from sklearn.ensemble import IsolationForest


def winsorization_outliers(df):
    out=[]
    for i in df:
        q1 = np.percentile(df , 1)
        q3 = np.percentile(df , 99)
        if i > q3 or i < q1:
            out.append(i)
    print("Outliers:",out)
    return out
    
outliers = winsorization_outliers(data.Age)

In [ ]:
data_age = data.loc[~data.Age.isin(outliers)]
sns.histplot(data=data_age, x="Age")

In [ ]:
import plotly.express as px

fig = px.violin(data_age, y="Age", x="mental_Health", color="Gender", box=True, points="all")
fig.show()